In [26]:
!pip install pandas pyarrow ollama --quiet


## Import Ollama and load Mistral

In [2]:
import ollama
import pandas as pd


In [3]:
print("loading data ...")
df = pd.read_parquet("../docs/cnn/test-00000-of-00001.parquet")
row = df.loc[0]
fulltext = row['article']
ref = row['highlights'] #human made ground truth
print(row['article'])
print("---")
print(row['highlights'])

loading data ...
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesd

In [5]:
print("summarizing ...")
response = ollama.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': f"generate a short summary of the following sentence:{fulltext}",
  },
])
gen = response['message']['content'] #generated output
print(gen)

summarizing ...
 The Palestinian Authority officially joined the International Criminal Court (ICC) on Wednesday as its 123rd member, granting the court jurisdiction over alleged crimes in Palestinian territories. The Palestinians signed the ICC's founding Rome Statute in January and accepted its jurisdiction over crimes committed since June 13, 2014. Israel and the United States opposed their entry into the body. Palestine Foreign Minister Riad al-Malki called it a move towards justice, while Judge Kuniko Ozaki emphasized the responsibilities that come with membership. Human Rights Watch welcomed Palestine's accession to the treaty. The ICC can now open a preliminary examination into alleged war crimes committed by both sides since June 2014. Israel and Hamas's conflict in Gaza last summer left over 2,000 people dead. The court was established in 2002 to prosecute genocide, crimes against humanity, and war crimes.


In [6]:
print (f"lenght of original full text: {len(fulltext)}")
print (f"lenght of generated summary : {len(gen)}")
print (f"lenght of human summary     : {len(ref)}")

lenght of original full text: 3612
lenght of generated summary : 912
lenght of human summary     : 233


## Let's use some metric to evaluate the result

In [7]:
!pip install rouge-score --quiet

## Let's create a wrong summary and a benchmark summary
The benchmark summary is created via chatgpt4

In [20]:
#the benchmark is generated manually on chatgpt4
bad = "The boat travels the seas, there is no point in telling this story now though"
ben = """
The Palestinian Authority became the 123rd member of the International Criminal Court (ICC) on Wednesday, gaining the court's jurisdiction over alleged crimes in Palestinian territories. This accession, marked by a ceremony in The Hague, follows the Palestinians' signing of the ICC's Rome Statute in January and accepting its jurisdiction since June 13, 2014. The move, which Israel and the U.S. oppose, allows for potential war crimes investigations against Israelis, but also subjects Palestinians to possible counter-charges. Palestinian Foreign Minister Riad al-Malki hailed the step towards justice and peace, while the ICC emphasized the responsibilities that come with membership. Human Rights Watch supported Palestine's ICC membership, criticizing pressures against it. The ICC's preliminary examination, opened in January, may lead to formal investigations of both sides for alleged war crimes, including those committed during the 2014 Gaza conflict.
"""


In [10]:
from rouge_score import rouge_scorer


In [13]:

def calculateRougeScore(metric,gen,ref,ben,bad):
    print(f"METRIC {metric}")

    rscorer = rouge_scorer.RougeScorer([metric], use_stemmer=True)
    sgener = rscorer.score ( ref , gen )[metric][2]
    sbench  = rscorer.score( ref , ben )[metric][2]
    sbad   = rscorer.score ( ref , bad )[metric][2]

    return {
        "metric": metric,
        "gen": sgener,
        "ben": sbench,
        "bad": sbad
    }

In [24]:
rouge3 = calculateRougeScore("rouge3",gen,ref,ben,bad=bad)
print (rouge3)

scoreframe = pd.DataFrame([{
    "metric":"rouge3",
    "our model summary":rouge3["gen"],
    "gpt-4 summary":rouge3["ben"],
    "random words summary":rouge3["bad"]
}],index=None)
scoreframe


METRIC rouge3
{'metric': 'rouge3', 'gen': 0.08187134502923975, 'ben': 0.09248554913294799, 'bad': 0.0}


,metric,our model summary,gpt-4 summary,random words summary
0,rouge3,0.081871,0.092486,0.0


In [25]:

rougeL = calculateRougeScore("rougeL",gen,ref,ben,bad=bad)
print (rougeL)

scoreframe.loc[len(scoreframe.index)] = ["rougeL",rougeL["gen"],rougeL["ben"],rougeL["bad"]]
scoreframe

METRIC rougeL
{'metric': 'rougeL', 'gen': 0.2742857142857143, 'ben': 0.22598870056497178, 'bad': 0.0816326530612245}


,metric,our model summary,gpt-4 summary,random words summary
0,rouge3,0.081871,0.092486,0.000000
1,rougeL,0.274286,0.225989,0.081633


In [26]:
from nltk.translate.bleu_score import sentence_bleu


In [28]:

print(f"------- BLEU ---------")
reference = [tuple(ref.split())]
candidate = [tuple(gen.split())]
benchmark = [tuple(ben.split())]
badsplit  = [tuple(bad.split())]

sgener = sentence_bleu(reference, candidate)
sbench = sentence_bleu(reference, benchmark)
sbad   = sentence_bleu(reference, badsplit)

print(f"gen:{sgener}")
print(f"ben:{sbench}")
print(f"bad:{sbad}")
scoreframe.loc[len(scoreframe.index)] = ["bleu",sgener,sbench,sbad]
scoreframe

------- BLEU ---------
gen:0
ben:0
bad:0


,metric,our model summary,gpt-4 summary,random words summary
0,rouge3,0.081871,0.092486,0.000000
1,rougeL,0.274286,0.225989,0.081633
2,bleu,0.000000,0.000000,0.000000


In [30]:
from nltk.translate import meteor_score
from nltk import word_tokenize

In [32]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/davide/nltk_data...


True

In [34]:
print(f"------- METEOR ---------")
tokenized_reference = word_tokenize(ref)
tokenized_candidate = word_tokenize(gen)
tokenized_benchmark = word_tokenize(ben)
tokenized_bad = word_tokenize(bad)

sgener = meteor_score.meteor_score([tokenized_reference], tokenized_candidate)
sbench = meteor_score.meteor_score([tokenized_reference], tokenized_benchmark)
sbad   = meteor_score.meteor_score([tokenized_reference], tokenized_bad)
print(f"gen:{sgener}")
print(f"ben:{sbench}")
print(f"bad:{sbad}")

scoreframe.loc[len(scoreframe.index)] = ["meteor",sgener,sbench,sbad]
scoreframe

------- METEOR ---------
gen:0.4369866529774128
ben:0.3431605909835823
bad:0.08595988538681948


,metric,our model summary,gpt-4 summary,random words summary
0,rouge3,0.081871,0.092486,0.000000
1,rougeL,0.274286,0.225989,0.081633
2,bleu,0.000000,0.000000,0.000000
3,meteor,0.436987,0.343161,0.085960


In [37]:
from openai import OpenAI
MODEL = "gpt-4"
api_key=open("../.openai").read()
client = OpenAI(api_key=api_key)
print("api_key:"+api_key[0:6]+"..."+api_key[-5:-1])

api_key:sk-CMu...yGwf


In [38]:
# Evaluation prompt template based on G-Eval
EVALUATION_PROMPT_TEMPLATE = """
You will be given one summary written for an article. Your task is to rate the summary on one metric.
Please make sure you read and understand these instructions very carefully. 
Please keep this document open while reviewing, and refer to it as needed.

Evaluation Criteria:

{criteria}

Evaluation Steps:

{steps}

Example:

Source Text:

{document}

Summary:

{summary}

Evaluation Form (scores ONLY):

- {metric_name}
"""

# Metric 1: Relevance

RELEVANCY_SCORE_CRITERIA = """
Relevance(1-5) - selection of important content from the source. \
The summary should include only important information from the source document. \
Annotators were instructed to penalize summaries which contained redundancies and excess information.
"""

RELEVANCY_SCORE_STEPS = """
1. Read the summary and the source document carefully.
2. Compare the summary to the source document and identify the main points of the article.
3. Assess how well the summary covers the main points of the article, and how much irrelevant or redundant information it contains.
4. Assign a relevance score from 1 to 5.
"""

# Metric 2: Coherence

COHERENCE_SCORE_CRITERIA = """
Coherence(1-5) - the collective quality of all sentences. \
We align this dimension with the DUC quality question of structure and coherence \
whereby "the summary should be well-structured and well-organized. \
The summary should not just be a heap of related information, but should build from sentence to a\
coherent body of information about a topic."
"""

COHERENCE_SCORE_STEPS = """
1. Read the article carefully and identify the main topic and key points.
2. Read the summary and compare it to the article. Check if the summary covers the main topic and key points of the article,
and if it presents them in a clear and logical order.
3. Assign a score for coherence on a scale of 1 to 5, where 1 is the lowest and 5 is the highest based on the Evaluation Criteria.
"""

# Metric 3: Consistency

CONSISTENCY_SCORE_CRITERIA = """
Consistency(1-5) - the factual alignment between the summary and the summarized source. \
A factually consistent summary contains only statements that are entailed by the source document. \
Annotators were also asked to penalize summaries that contained hallucinated facts.
"""

CONSISTENCY_SCORE_STEPS = """
1. Read the article carefully and identify the main facts and details it presents.
2. Read the summary and compare it to the article. Check if the summary contains any factual errors that are not supported by the article.
3. Assign a score for consistency based on the Evaluation Criteria.
"""

# Metric 4: Fluency

FLUENCY_SCORE_CRITERIA = """
Fluency(1-3): the quality of the summary in terms of grammar, spelling, punctuation, word choice, and sentence structure.
1: Poor. The summary has many errors that make it hard to understand or sound unnatural.
2: Fair. The summary has some errors that affect the clarity or smoothness of the text, but the main points are still comprehensible.
3: Good. The summary has few or no errors and is easy to read and follow.
"""

FLUENCY_SCORE_STEPS = """
Read the summary and evaluate its fluency based on the given criteria. Assign a fluency score from 1 to 3.
"""


In [39]:
def get_geval_score(
    criteria: str, steps: str, document: str, summary: str, metric_name: str
):
    prompt = EVALUATION_PROMPT_TEMPLATE.format(
        criteria=criteria,
        steps=steps,
        metric_name=metric_name,
        document=document,
        summary=summary,
    )
    
    response = client.chat.completions.create(

        model = MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=5,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
    )
    return response.choices[0].message.content



In [40]:
evaluation_metrics = {
    "Relevance": (RELEVANCY_SCORE_CRITERIA, RELEVANCY_SCORE_STEPS),
    "Coherence": (COHERENCE_SCORE_CRITERIA, COHERENCE_SCORE_STEPS),
    "Consistency": (CONSISTENCY_SCORE_CRITERIA, CONSISTENCY_SCORE_STEPS),
    "Fluency": (FLUENCY_SCORE_CRITERIA, FLUENCY_SCORE_STEPS),
}

In [42]:


summaries = {"our model summary": gen, "gpt-4 summary": ben, "random words":bad}

data = {"Evaluation Type": [], "Summary Type": [], "Score": []}

for eval_type, (criteria, steps) in evaluation_metrics.items():
    for summ_type, summary in summaries.items():
        data["Evaluation Type"].append(eval_type)
        data["Summary Type"].append(summ_type)
        result = get_geval_score(criteria, steps, fulltext, summary, eval_type)
        score_num = int(result.strip())
        data["Score"].append(score_num)

pivot_df = pd.DataFrame(data, index=None).pivot(
    index="Evaluation Type", columns="Summary Type", values="Score"
)
pivot_df

Summary Type,gpt-4 summary,our model summary,random words
Evaluation Type,,,
Coherence,5,5,1
Consistency,5,5,1
Fluency,3,3,1
Relevance,5,5,1


In [45]:
scoreframe

,metric,our model summary,gpt-4 summary,random words summary
0,rouge3,0.081871,0.092486,0.000000
1,rougeL,0.274286,0.225989,0.081633
2,bleu,0.000000,0.000000,0.000000
3,meteor,0.436987,0.343161,0.085960
